In [104]:
import numpy as np

## TELECOM ##

#Variabili

## Trasmettitore
P_t = 35 # Potenza (dBm)
Total_Loss_t = 3 # Perdite totali (dB)
G_t = 15  # Gain TX
Freq_t = 21  # Frequenza di trasmissione (GHz)
B_Width_t = 1 # Larghezza di banda (MHz)
alpha_t = 1 # roll-off
Cone_angle = 40 # degrees
####

## Ricevitore

G_r = 15  # Gain RX
T_s = 400  # Total noise temperature (K)
Freq_r = 18  # Frequenza ricezione (GHz)
B_Width_r = 1  # Larghezza di banda (MHz)
Total_Loss_r = 4  # perdite totali RX
alpha_r = 1 # roll-off

####
## Sistema

POW = 20  # Consumo in (W)
POW_dissipation = 10 # in (W)
Mass = 2  #Massa (kg)
Functioning_Range = [-30, 80] # [Min, Max] in (°C)
L_l = 1 #Line loss (dB)
# R =   # Data rate
h = 750  # altezza orbita (km)
# L_a =   # Path loss
Vol = 30*10*10  # Volume (cm^3)
payload = 50*24*(2000*2000) #50 immagini a 24 bit per 2000x2000 pixel (bit)
commands = 5 # istruzioni dalla gnd (Gbit)
####
## Ground Station

bandaStazione = "K" ## 18-26 GHz
GS_r = 40 # gain antemma (dBi)
GS_t = 40 # gain antemma (dBi)
GS_TX_power = 50 # potenza trasmissione (dBm)
GS_Total_Loss_r = 2 # (dB)
total_NoiseTemp = 600*1000 # (K)*1000 per la conversione in dBm

#E_b =   # Energy per bit
k = 1.380649*10**(-23) # Costante di Boltzmann (J/K)
k_dB = -228.6 # Costante di Boltzmann (dB J/K)
c = 2.997e+5 # Velocità della luce (km/s)
 # Noise density
caricaProprietà =1

In [105]:
caricaProprietà =1

class RX:
    def __init__(self, banda, frequenza, rate, gain, alfa, tot_losses, total_noise_temp):
        self.banda = banda
        self.frequenza = frequenza
        self.alfa = alfa
        self.total_noise_temp = total_noise_temp
        self.tot_losses = tot_losses
        self.rate = rate #da ricavare
        self.gain = gain #da ricavare

class TX:
    def __init__(self, banda, frequenza, rate, gain, alfa, tot_losses, power, coverage, EIRP=0):
        self.M = 8
        self.banda = banda
        self.frequenza = frequenza
        self.power = power 
        self.tot_losses = tot_losses
        self.alfa = alfa
        self.gain = gain #da ricavare
        self.rate = rate
        self.coverage = coverage # angolo di copertura (40° su STK)
        self.EIRP = EIRP




class Basic:
    def __init__(self, massa, T_min, T_max, power, power_dissipation, volume):
        self.massa = massa
        self.T_min = T_min
        self.T_max = T_max
        self.power = power
        self.power_dissipation = power_dissipation
        self.volume = volume

class S_C:
    def __init__(self):
        self.rx = RX(banda=0, frequenza=0, rate=0, gain=0, alfa=0, tot_losses=0, total_noise_temp=0)
        self.tx = TX(banda=0, frequenza=0, rate=0, gain=0, alfa=0, tot_losses=0, power=0, coverage=0)
        self.basic = Basic(massa=0, T_min=0, T_max=0, power=0, power_dissipation=0, volume=0)

class G_S:
    def __init__(self, banda='', G_r=0, G_t=0, power_t=0, total_Loss_t=0, total_noiseTemp=0):
        self.banda = banda
        self.gain_r = G_r
        self.gain_t = G_t
        self.power = power_t
        self.Loss_t = total_Loss_t
        self.noiseTemp = total_noiseTemp


def caricaRX(S_C, banda, frequenza, gain, alfa, tot_losses, total_noise_temp, rate=0):
    S_C.rx.banda = banda
    S_C.rx.frequenza = frequenza
    S_C.rx.rate = rate
    S_C.rx.gain = gain
    S_C.rx.alfa = alfa
    S_C.rx.tot_losses = tot_losses
    S_C.rx.total_noise_temp = total_noise_temp
    return S_C

def caricaTX(S_C, banda, frequenza, gain, alfa, tot_losses, power, coverage, rate=0):
    S_C.tx.banda = banda*10**6
    S_C.tx.frequenza = frequenza*(10**9)
    S_C.tx.rate = calcola_Rate(banda*10**6, alfa, S_C.tx.M)
    S_C.tx.gain = gain
    S_C.tx.alfa = alfa
    S_C.tx.tot_losses = tot_losses
    S_C.tx.power = power
    S_C.tx.coverage = coverage
    S_C.tx.EIRP = calcola_EIRPdB(power, tot_losses, gain)
    return S_C

def caricaBasic(S_C, massa, T_min, T_max, power, power_dissipation, volume, payload, commands):
    S_C.basic.massa = massa
    S_C.basic.T_min = T_min
    S_C.basic.T_max = T_max
    S_C.basic.power = power
    S_C.basic.power_dissipation = power_dissipation
    S_C.basic.volume = volume
    S_C.basic.payload = payload
    S_C.basic.commands = commands
    return S_C

def caricaGroundStation(G_S, banda, G_r, G_t, power_t, total_Loss_t, total_noiseTemp):
    G_S.banda = banda
    G_S.gain_r = G_r
    G_S.gain_t = G_t
    G_S.power = power_t
    G_S.Loss_t = total_Loss_t
    G_S.noiseTemp = total_noiseTemp
    return G_S

def calcola_EIRPdB(Ptx, L_l, G_t):
    return Ptx-L_l+G_t

def calcola_E_bdB(EIRP, L_s, L_a, G_r, R):
    return EIRP + L_s + L_a + G_r - R

def calcola_NoiseDensity(k, T_s):
    return k*T_s

def calcola_Loss_SpacedB(distance, frequenza):
    wavelenght = c/(frequenza)
    return 10*np.log10((4*np.pi*distance/wavelenght)**2)

def calcola_Loss_AtmosfericadB(pioggia):
    if pioggia == 1:
        return 3.5 #pioggia
    else:                       # entrambi +0.5 per l'assorbimento da gas allo zenith (che è la nostra approssimazione)
        return 1 #clear sk

def power_ReceivedAtAntenna(EIRP, total_PATH_Losses, ReceiverGain, Receiver_Losses):
    return EIRP - total_PATH_Losses + ReceiverGain - Receiver_Losses

def E0_over_N0dB(EIRP, Ls, La, Gr, Ts, R):
    print(f"EIRP = {EIRP}, SpaceLoss = {Ls}, AtmoLoss = {La} ; GSgain = {Gr} ; GSnoiseT = {Ts} ; SCrate {R} ; k_boltz {k}")
    return EIRP - Ls - La + Gr - 10*np.log10(k*Ts) - 10*np.log10(R)

def calcola_Rate(B, alfa, M):
    return B/(alfa)*np.log2(M)


In [106]:
# Esempio di utilizzo delle classi
s_c = S_C()
# carica proprietà
pioggia=1
s_c = caricaRX(s_c, B_Width_r, Freq_r, G_r, alpha_r, Total_Loss_r, T_s)
s_c = caricaTX(s_c, B_Width_t, Freq_t, G_t, alpha_t, Total_Loss_t, P_t, Cone_angle)
s_c = caricaBasic(s_c, Mass, Functioning_Range[0], Functioning_Range[1], POW, POW_dissipation, Vol, payload, commands)
g_s = G_S()
g_s = caricaGroundStation(g_s, bandaStazione,GS_r, GS_t, GS_TX_power, GS_Total_Loss_r, total_NoiseTemp)
print(s_c.tx.rate)
LS = calcola_Loss_SpacedB(h, s_c.tx.frequenza)
LA = calcola_Loss_AtmosfericadB(pioggia)
EEE=E0_over_N0dB(s_c.tx.EIRP, calcola_Loss_SpacedB(h, s_c.tx.frequenza), calcola_Loss_AtmosfericadB(pioggia), g_s.gain_r, g_s.noiseTemp, s_c.tx.rate)
print(f"EIRP = {s_c.tx.EIRP}, SpaceLoss = {LS}, AtmoLoss = {LA} ; GSgain = {g_s.gain_r} ; GSnoiseT = {g_s.noiseTemp} ; SCrate {s_c.tx.rate}")

a = calcola_Loss_SpacedB(h, s_c.tx.frequenza)
print(EEE)


3000000.0
EIRP = 47, SpaceLoss = 176.3960735840414, AtmoLoss = 3.5 ; GSgain = 40 ; GSnoiseT = 600000 ; SCrate 3000000.0 ; k_boltz 1.380649e-23
EIRP = 47, SpaceLoss = 176.3960735840414, AtmoLoss = 3.5 ; GSgain = 40 ; GSnoiseT = 600000 ; SCrate 3000000.0
13.150368538143198


In [107]:
print(s_c.tx.rate)
0.48/120/(s_c.tx.rate *1e-9) #(Gb/sec)

5e+8 / s_c.tx.rate

3000000.0


166.66666666666666